In [1]:
! pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=47058d34222494f26cc21f30b8de53bb22e4b40e6e39de64391a7c992f12412b
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
spark = SparkSession.builder.appName("firstapp").getOrCreate()

Question 1: calculate the total cost for all customers and all subscribers

In [3]:
df = spark.read.csv("/content/2017-fordgobike-tripdataa (1).csv", header = True,inferSchema = True)

In [4]:
df.show(5)

+----------+---------------+-----------------+------------------+---+--------+-------------+---------------+----------------+---+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+-----------+
|start_time|start time hour|start time minute|start time seconds|_c4|end_time|end_time hour|end_time minute|end_time seconds|_c9|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude|bike_id| user_type|member_birth_year|member_gender|     pyment|
+----------+---------------+-----------------+------------------+---+--------+-------------+---------------+----------------+---+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------

In [5]:
df.columns

['start_time',
 'start time hour',
 'start time minute',
 'start time seconds',
 '_c4',
 'end_time',
 'end_time hour',
 'end_time minute',
 'end_time seconds',
 '_c9',
 'start_station_id',
 'start_station_name',
 'start_station_latitude',
 'start_station_longitude',
 'end_station_id',
 'end_station_name',
 'end_station_latitude',
 'end_station_longitude',
 'bike_id',
 'user_type',
 'member_birth_year',
 'member_gender',
 'pyment']

In [7]:
#from wweek2.tasks, we calculated the trip cost, as shown below
df1 = df.withColumn('start_time(s)', df[1]*3600 + df[2]*60 + df[3])
df2 = df1.withColumn('end_time(s)', df[6]*3600 + df[7]*60 + df[8])
df3 = df2.withColumn('duration for each trip(s)' , (df2['end_time(s)'] - df2['start_time(s)']))
df4 = df3.withColumn('trip_cost', ((df3['duration for each trip(s)']/60)*0.35))
df5 = df4.filter(df4['trip_cost']>0)
df5.select('trip_cost').show(10)

+------------------+
|         trip_cost|
+------------------+
|14.985833333333334|
|110.67583333333332|
|2.3566666666666665|
|25.281666666666666|
|24.208333333333332|
|           24.7275|
|19.209166666666665|
|18.538333333333334|
|12.734166666666665|
|12.664166666666665|
+------------------+
only showing top 10 rows



In [8]:
df5.groupBy('user_type').agg({'trip_cost': 'sum'}).show()

+----------+------------------+
| user_type|    sum(trip_cost)|
+----------+------------------+
|Subscriber| 388962.0524999987|
|  Customer|171938.80666666647|
+----------+------------------+

